In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.arima.model import ARIMA


In [10]:
# Load data
data = pd.read_csv("../src/data/merged_data.csv", parse_dates=["Date"], index_col="Date")

In [11]:
# --------------------------
# 1. Data Preprocessing
# --------------------------
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Split data into train and test
train_size = int(len(data) * 0.8)
train, test = data_scaled.iloc[:train_size], data_scaled.iloc[train_size:]


In [12]:
# --------------------------
# 2. ARIMA Model
# --------------------------
arima_model = ARIMA(train["Price"], order=(5,1,0)).fit()
arima_pred = arima_model.forecast(steps=len(test))

c:\Users\Administrator.MMCY\OneDrive - MMCYTECH\Desktop\10a\Birhan-Energies-W10\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator.MMCY\OneDrive - MMCYTECH\Desktop\10a\Birhan-Energies-W10\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator.MMCY\OneDrive - MMCYTECH\Desktop\10a\Birhan-Energies-W10\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator.MMCY\OneDrive - MMCYTECH\Desktop\10